# Diccionarios (HashMap)

**Curso:** Estructuras de Datos (Ingeniería de Sistemas)

**Propósito:** Comprender y aplicar las ideas clave del tema, analizando complejidad y usos prácticos.

### Objetivos de aprendizaje
- Reconocer el modelo de datos y operaciones fundamentales.
- Analizar la complejidad temporal y espacial de las operaciones clave.
- Implementar y probar funciones en Python con casos de uso reales.
- Comparar ventajas y limitaciones frente a alternativas.


### Teoría esencial

Mapeo clave→valor basado en tabla hash. 
Operaciones promedio O(1) (peor caso O(n)). Claves deben ser hashables/inmutables.

### Guía de estudio
- Identifica operaciones fundamentales y su costo asintótico.
- Observa invariante(s) de la estructura (lo que siempre debe cumplirse).
- Relaciona la estructura con patrones de uso en software (colas de trabajo, caches, planificadores, etc.).

### Implementación base (desde el archivo `.py`)
El siguiente bloque contiene el código original.

In [ ]:
print('*** Sistema de Inventarios ***')

inventario = []

numero_productos = int(input('Cuantos productos deseas agregar al inventario? '))

for indice in range(numero_productos):
    print(f'Proporciona los valores del producto {indice + 1}')
    nombre = input('Nombre: ')
    precio = float(input('Precio: '))
    cantidad = int(input('Cantidad: '))
    # Creamos el diccionario con el detalle del producto
    producto = {'id': indice, 'nombre': nombre, 'precio': precio, 'cantidad': cantidad}
    # Agregamos el nuevo producto al inventario
    inventario.append(producto)

# Mostrar el inventario inicial
print(f'\nInventario inicial: {inventario}')

### Pruebas rápidas
Usa estos ejemplos para verificar comportamientos básicos. Ajusta según tus funciones.

In [ ]:
"""Pruebas inline (sin modificar el código original de captura por input).
Creamos funciones utilitarias NUEVAS sobre la misma estructura de datos
(list[dict]) para poder probar operaciones de inventario.
Si existe pytest en el entorno, también se ejecuta una mini batería.
"""
from typing import List, Dict, Any, Optional, Callable
from time import perf_counter
import math

Producto = Dict[str, Any]
Inventario = List[Producto]

# -------------------------- FUNCIONES (no alteran el código original) -------------------------- #

def add_product(inv: Inventario, id_: int, nombre: str, precio: float, cantidad: int,
                categoria: str = 'general', activo: bool = True, tags: Optional[List[str]] = None) -> bool:
    if any(p['id'] == id_ for p in inv):  # id único
        return False
    if precio < 0 or cantidad < 0:
        raise ValueError("Precio y cantidad deben ser >= 0")
    inv.append({
        'id': id_, 'nombre': nombre, 'precio': float(precio), 'cantidad': int(cantidad),
        'categoria': categoria, 'activo': bool(activo), 'tags': tags or []
    })
    return True


def find_by_id(inv: Inventario, id_: int) -> Optional[Producto]:
    for p in inv:
        if p['id'] == id_:
            return p
    return None


def find_by_nombre(inv: Inventario, nombre: str) -> List[Producto]:
    return [p for p in inv if p['nombre'].lower() == nombre.lower()]


def update_stock(inv: Inventario, id_: int, delta: int) -> bool:
    p = find_by_id(inv, id_: int)
    if p is None:
        return False
    nuevo = p['cantidad'] + delta
    if nuevo < 0:
        return False
    p['cantidad'] = nuevo
    return True


def valor_total(inv: Inventario) -> float:
    return sum(p['precio'] * p['cantidad'] for p in inv)


def low_stock(inv: Inventario, umbral: int) -> List[Producto]:
    return [p for p in inv if p['cantidad'] <= umbral]


def remove_product(inv: Inventario, id_: int) -> bool:
    for i, p in enumerate(inv):
        if p['id'] == id_:
            del inv[i]
            return True
    return False


def build_index(inv: Inventario) -> Dict[int, Producto]:
    return {p['id']: p for p in inv}


def update_stock_index(idx: Dict[int, Producto], id_: int, delta: int) -> bool:
    p = idx.get(id_)
    if p is None:
        return False
    nuevo = p['cantidad'] + delta
    if nuevo < 0:
        return False
    p['cantidad'] = nuevo
    return True

# ---- Edge case: simular colisiones (wrapper de id con hash forzado) ---- #
class IdColision:
    __slots__ = ('valor',)
    def __init__(self, valor: int):
        self.valor = valor
    def __hash__(self):  # Fuerza colisiones devolviendo hash constante
        return 42
    def __eq__(self, other):
        return isinstance(other, IdColision) and self.valor == other.valor
    def __repr__(self):
        return f"IdColision({self.valor})"

# -------------------------- PRUEBAS INLINE -------------------------- #

def _run_inline_tests():
    inv: Inventario = []
    assert add_product(inv, 1, 'Teclado', 50_000, 10)
    assert add_product(inv, 2, 'Mouse', 30_000, 5)
    assert not add_product(inv, 1, 'Duplicado', 1, 1)  # id repetido
    assert math.isclose(valor_total(inv), 50_000*10 + 30_000*5)
    assert update_stock(inv, 2, +3)
    assert find_by_id(inv, 2)['cantidad'] == 8
    assert not update_stock(inv, 2, -100)  # no permite negativo
    assert remove_product(inv, 1)
    assert find_by_id(inv, 1) is None
    # índice
    add_product(inv, 10, 'Monitor', 500_000, 2)
    add_product(inv, 11, 'Cámara', 250_000, 1)
    idx = build_index(inv)
    assert update_stock_index(idx, 10, +1)
    assert idx[10]['cantidad'] == 3
    # Low stock
    bajos = low_stock(inv, 2)
    ids_bajos = {p['id'] for p in bajos}
    assert 11 in ids_bajos and 10 not in ids_bajos
    # Búsqueda nombre (case-insensitive)
    assert find_by_nombre(inv, 'monitor')[0]['id'] == 10
    print("✅ Pruebas inline: OK (CRUD, valor total, índice, low stock, búsqueda nombre, integridad)")

_run_inline_tests()

# -------------------------- MINI BENCHMARK -------------------------- #

def _benchmark(n: int = 10_000):
    inv: Inventario = []
    for i in range(n):
        add_product(inv, i, f'Prod{i}', 100.0, 10)
    # Búsqueda lineal
    objetivo = n - 5
    t0 = perf_counter()
    _ = find_by_id(inv, objetivo)
    t_lineal = perf_counter() - t0
    # Índice
    t1 = perf_counter()
    idx = build_index(inv)
    _ = idx.get(objetivo)
    t_index_total = perf_counter() - t1  # incluye construcción
    # Solo lookup con índice
    t2 = perf_counter()
    _ = idx.get(objetivo)
    t_lookup = perf_counter() - t2
    return {
        'n': n,
        'lineal_us': t_lineal * 1e6,
        'index_build+lookup_us': t_index_total * 1e6,
        'index_lookup_only_us': t_lookup * 1e6
    }

bench = _benchmark(5_000)
print(f"Benchmark n={bench['n']}: lineal={bench['lineal_us']:.2f}µs | build+lookup={bench['index_build+lookup_us']:.2f}µs | lookup_only={bench['index_lookup_only_us']:.2f}µs")

# -------------------------- CASO LÍMITE (COLISIONES) -------------------------- #

def _colisiones_demo():
    tabla = {}
    claves = [IdColision(i) for i in range(50)]
    for k in claves:
        tabla[k] = k.valor
    # Acceso: en CPython aún es rápido pero conceptualmente todos comparten mismo bucket lógico
    acc_ok = all(tabla[c] == c.valor for c in claves)
    return acc_ok, len(tabla)

ok, tam = _colisiones_demo()
print(f"Colisiones simuladas: correcto={ok}, elementos={tam}")

# -------------------------- PYTEST (opcional) -------------------------- #
try:
    import pytest  # type: ignore
    def test_add_product():
        inv = []
        assert add_product(inv, 1, 'A', 10, 1)
        assert not add_product(inv, 1, 'B', 5, 2)
    def test_valor_total():
        inv = []
        add_product(inv, 1, 'A', 10, 2)
        add_product(inv, 2, 'B', 5, 4)
        assert valor_total(inv) == 10*2 + 5*4
    def test_update_stock():
        inv = []
        add_product(inv, 3, 'C', 1, 5)
        assert update_stock(inv, 3, -2)
        assert find_by_id(inv, 3)['cantidad'] == 3
        assert not update_stock(inv, 3, -10)
    def test_low_stock():
        inv = []
        add_product(inv, 1, 'A', 1, 1)
        add_product(inv, 2, 'B', 1, 10)
        assert {p['id'] for p in low_stock(inv, 2)} == {1}
    def test_index_update():
        inv = []
        add_product(inv, 1, 'A', 1, 5)
        idx = build_index(inv)
        assert update_stock_index(idx, 1, +2)
        assert idx[1]['cantidad'] == 7
    print("(pytest disponible: puedes ejecutar estas pruebas externamente)")
except ImportError:
    print("(pytest no disponible: solo pruebas inline ejecutadas)")

### Complejidad (análisis informal)
Completa esta tabla de forma crítica, justificando cada costo.

| Operación | Mejor caso | Promedio | Peor caso | Nota |
|---|---|---|---|---|
| op1 | O( ) | O( ) | O( ) |  |
| op2 | O( ) | O( ) | O( ) |  |
| op3 | O( ) | O( ) | O( ) |  |

### Tabla de complejidad (completada)
| Operación | Mejor | Promedio | Peor | Nota |
|-----------|-------|----------|------|------|
| add_product (sin índice) | O(n) (chequeo id) | O(n) | O(n) | Busca duplicado lineal |
| find_by_id (sin índice) | O(1) | O(n/2) | O(n) | Lineal |
| find_by_id (con índice hash) | O(1) | O(1) | O(n) | Peor caso colisiones extremas |
| build_index | O(n) | O(n) | O(n) | Inserciones hash |
| update_stock (sin índice) | O(n) | O(n) | O(n) | Incluye búsqueda |
| update_stock_index | O(1) | O(1) | O(n) | Hash degenerado |
| valor_total | O(n) | O(n) | O(n) | Recorrido completo |
| low_stock | O(n) | O(n) | O(n) | Filtrado |
| remove_product | O(1) | O(n/2) | O(n) | Búsqueda+desplazamiento |
| build_index + lookup masivo k | O(n+k) | O(n+k) | O(nk) | Si tabla colapsa (teórico) |
| Colisiones simuladas | O(1) | O(1) | O(n) | Hash constante |

Observación: En práctica CPython maneja bien colisiones, pero teóricamente degradan a O(n).

### Ejercicios propuestos
1. Implementa pruebas unitarias con `pytest` para al menos 5 funciones/operaciones.
2. Mide empíricamente tiempos (con `timeit`) al variar n, y compara con el análisis teórico.
3. Extiende la implementación para cubrir un caso límite (overflow, colisión, degeneración, etc.).
4. Escribe una breve reflexión: ¿en qué contextos reales usarías esta estructura sobre sus alternativas?

### Ejercicios propuestos (resueltos sin alterar código original)
1. Pruebas unitarias: implementadas como pruebas inline + opcional pytest si existe.
2. Medición tiempos: mini benchmark compara búsqueda lineal vs índice hash.
3. Caso límite: clase IdColision fuerza hash constante para mostrar degradación conceptual.
4. Reflexión:
   - Diccionario es ideal para acceso directo por id cuando se superan ~O(log n) estructuras (listas ordenadas).
   - Para gran volumen + consultas complejas: bases de datos o estructuras avanzadas (árboles B, tries).
   - Listas lineales adecuadas solo para datasets muy pequeños o escenarios donde el orden de inserción es crítico y el costo de índice no compensa.
   - El patrón build_index permite trade-off tiempo de preparación vs consultas masivas.


### Referencias rápidas
- Cormen, Leiserson, Rivest, Stein. *Introduction to Algorithms* (CLRS).
- Sedgewick & Wayne. *Algorithms*.
- Documentación oficial de Python: `collections`, `heapq`, `bisect`, `array`.

*Fuente de código:* `14-Solucion-SistemaInventario-UP.py`  — Generado automáticamente el 2025-09-08 22:41.